##Install library

In [32]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_applications
!pip install mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-rcsdzt5s
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-rcsdzt5s
  Resolved https://github.com/rcmalli/keras-vggface.git to commit bee35376e76e35d00aeec503f2f242611a97b38a
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Import library

In [33]:
import tensorflow as tf
import keras
import keras_vggface
from keras_vggface.vggface import VGGFace
import mtcnn
from mtcnn.mtcnn import MTCNN
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.utils.data_utils import get_file
import keras_vggface.utils
import PIL
from PIL import Image
import os
import cv2
from google.colab.patches import cv2_imshow

##Function to convert video to jpg per frame

In [34]:
# Function to extract frames
def frameCapture(path):
  
    # Path to video file
    vidObj = cv2.VideoCapture(path)
  
    # Used as counter variable
    count = 0
  
    # checks whether frames were extracted
    success = True
  
    while success:
  
        # vidObj object calls read
        # function extract frames
        success, image = vidObj.read()
        
        if success == False:
          break

        # Saves the frames with frame-count
        cv2.imwrite("/content/obama4/frame%d.jpg" % count, image)
  
        count += 1

##Crop using Haar Cascade Classifier

In [35]:
haar_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [36]:
def crop_haar(img):
  img = cv2.imread(img)
  gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  detected = haar_classifier.detectMultiScale(gimg)

  if detected == ():
    return None
  else:
    (x, y, w, h) = detected[0]
    cimg = img[y:y+h , x:x+w]
    cv2.resize(cimg, (224, 224))
    cv2_imshow(cimg)
    # return cimg

In [37]:
# crop_haar("/content/momo_3.jpg")

##Crop using MTCNN

In [38]:
def read_image(file):
  img = cv2.imread(file)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

def crop_bb(image, detection, margin):
    x1, y1, w, h = detection['box']
    x1 -= margin
    y1 -= margin
    w += 2*margin
    h += 2*margin
    
    if x1 < 0:
        w += x1
        x1 = 0

    if y1 < 0:
        h += y1
        y1 = 0

    return image[y1:y1+h, x1:x1+w]

def crop_mtcnn(mtcnn, img):
  det = mtcnn.detect_faces(img)[0]
  margin = int(0.1 * img.shape[0])
  ret = crop_bb(img, det, margin)
  return ret

In [39]:
# import matplotlib.pyplot as plt

# momo1 = read_image("/content/drive/MyDrive/dataset/momo/1.jpg")
# momo2 = read_image("/content/drive/MyDrive/dataset/momo/2.jpg")
# momo3 = read_image("/content/drive/MyDrive/dataset/momo/3.jpg")

# #crops the faces
# mtcnn = MTCNN()
# momo1 = crop_mtcnn(mtcnn, momo1)
# momo2 = crop_mtcnn(mtcnn, momo2)
# momo3 = crop_mtcnn(mtcnn, momo3)

# fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
# fig.set_size_inches(16, 8)
# ax1.imshow(momo1)
# ax1.text(0.5,-0.1,"Momo 1", transform=ax1.transAxes)
# ax2.imshow(momo2)
# ax1.text(0.5,-0.1,"Momo 2", transform=ax2.transAxes)
# ax3.imshow(momo3)
# ax1.text(0.5,-0.1,"Momo 3", transform=ax3.transAxes)

# plt.show()

##Extractor

In [40]:
def extract_face(filename, required_size=(224, 224)):
  # load image from file
  pixels = plt.imread(filename)
  
  # create the detector, using default weights
  detector = MTCNN()
  
  # detect faces in the image
  results = detector.detect_faces(pixels)
  
  # extract the bounding box from the first face
  x1, y1, width, height = results[0]['box']
  x2, y2 = x1 + width, y1 + height
  
  # extract the face
  face = pixels[y1:y2, x1:x2]
  
  # resize pixels to the model size
  image = Image.fromarray(face)
  image = image.resize(required_size)
  face_array = np.asarray(image)
  return face_array

In [41]:
# #test function
# pixels = extract_face('/content/momo_3.jpg')
# # plot the extracted face
# plt.imshow(pixels)
# # show the plot
# plt.show()

##Load model VGGFace

In [42]:
pre_trained_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), weights='vggface')

In [43]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [44]:
pre_trained_model.summary()

Model: "vggface_resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                         )                                                  

In [48]:
last_layer = pre_trained_model.get_layer('activation_141')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 2048)


In [49]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model
from tensorflow.keras import layers

In [50]:
import glob

folders = glob.glob('/content/drive/MyDrive/dataset/image/*')
# Flatten the output layer to 1 dimension
x = layers.Conv2D(1024, (7,7), activation='relu')(last_output)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)   

# Flatten the output layer to 1 dimension
x = layers.Conv2D(1024, (1,1), activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)  

# Flatten the output layer to 1 dimension
x = layers.Conv2D(1024, (1,1))(x)

# Add a final sigmoid layer for classification
x = layers.Flatten()(x)

# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
                 
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

x = layers.Dense(len(folders), activation='softmax')

In [51]:
import glob
category = {}
folders = glob.glob('/content/drive/MyDrive/dataset/image/*')
for i, name in enumerate(folders):
  category[i] = name.split('/')[-1]

In [52]:
category

{0: 'ashel',
 1: 'shani',
 2: 'marsha',
 3: 'gracia',
 4: 'saka',
 5: 'kounde',
 6: 'frenkie',
 7: 'momo'}

In [53]:
model = Model(pre_trained_model.input, x) 

ValueError: ignored